In [9]:
import math
import numpy as np

In [10]:
#http://infolab.stanford.edu/~datar/courses/cs361a/papers/quantiles.pdf
#creates a summary on the data that gaurantees e-quantiles
def GK_summary (data, e):
    S = []
    s = 0
    for n in range (0,len(data)):
        if (n%(1/(2*e)) == 0 and s>4):
            #COMPRRESS()
            for i in range (s-4,0, -1):
                t_i = S[i]
                t_i_next = S[i+1]
                g_i_star  = t_i[1]
                descendents_list = []
                descendent_index = i - 1
                while ((BAND(S[descendent_index], n, e) < BAND(t_i, n, e)) and (descendent_index>=0)):
                    g_i_star += S[descendent_index][1]
                    descendents_list.append (descendent_index)
                    descendent_index -= 1 
                g_i_next = t_i_next[1]
                delta_i_next = t_i_next[2]
                if ((BAND(t_i, n, e)<= BAND (t_i_next, n, e)) and (g_i_star + g_i_next + delta_i_next <2*e*n)):
                    S[i+1][1]+= g_i_star
                    del S[i]
                    s-=1          
                    for j in range (len(descendents_list)):
                        del S[descendents_list[j]]
                        s-=1
        #INSERT
        v = data[n]
        if (len(S)==0):
            S.insert (0, [v, 1, 0])
            s+=1
        elif (v<S[0][0]):
            S.insert (0, [v, 1, 0])
            s+=1
        elif (v>S[len(S)-1][0]):
            S.append ([v, 1, 0])
            s+=1
        else:
            for i in range (1,len(S)):
                v_i_prior = S[i-1][0]
                v_i = S[i][0]
                if (v_i_prior<=v<v_i):
                    S.insert (i, [v, 1, math.floor(2*e*n)])
                    s+=1
                    break;  
    return S;

In [11]:
#From bands section
def BAND (t_i, n, e):
    #first 1/2e observations are in their own band
    if (n<(1/2*e)):
        return -1
    p = math.floor (2*e*n)
    if (t_i[2]==p):
        return 0
    band = 0
    for a in range (1,math.ceil(math.log(2*e*n))):
        if ((p-2**a-(p%2**a))<t_i[2]<=(p-2**(a-1)-(p%2**(a-1)))):
            band = a
            break;
    return band

In [12]:
#O((1/e)log(eN))
def GK_quantile (S, e, q, N):
    r = math.ceil(q*N)
    r_min_v_i = 0
    v_i = None
    for i in range (len(S)):
        r_min_v_i += S[i][1]
        r_max_v_i = r_min_v_i + S[i][2]
        if (((r-r_min_v_i)<=e*N) and ((r_max_v_i - r) <= e*N)):
            v_i =  S[i][0]
            break;
    return v_i